<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Preprocessing_CrowdFlower_using_Ekphrasis_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

In [2]:
!pip3 install ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import pandas as pd
import re, sys, os, csv
import io

     |████████████████████████████████| 81kB 3.4MB/s 
     |████████████████████████████████| 184kB 6.6MB/s 
     |████████████████████████████████| 71kB 4.6MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp36-none-any.whl size=82844 sha256=7f42ea62962ded36118606fdfa7eadb77857b3416a717cb7a1a5f6c51910c270
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp36-none-any.whl size=46451 sha256=ec332b07c7623ffc08466d83b33f66d713af75d38082b13adb9b401b12294ae0
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy


In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import re, sys, os, csv
from numpy import array
from numpy import asarray
from numpy import zeros
import itertools
!pip3 install emoji
import emoji
from nltk.stem.porter import PorterStemmer



#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

import pickle

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
SEED = 42
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

import keras
import keras.utils
from keras import utils as np_utils
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras import backend as K

from keras.preprocessing import text,sequence
from keras.layers import Input
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Lambda, BatchNormalization, Concatenate
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
import matplotlib.pyplot as plt
from collections import Counter
import keras.layers as layers
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.engine import Layer
from keras.layers import TimeDistributed
from keras.layers import Activation, Flatten

sess = tf.Session()
# K.set_session(sess)

     |████████████████████████████████| 133kB 5.3MB/s 
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
import nltk
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [5]:
from nltk.metrics.distance import (edit_distance,jaccard_distance)
from nltk.util import ngrams

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Methods for Cleaning

In [7]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
}

In [8]:
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
	"cant":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
	"didnt":"did not",
        "doesn't":"does not",
	"doesnt":"does not",
        "don't":"do not",
	"dont":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
	"hadnt":"had not",
        "hasnt":"has not",
        "havent":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
	"Im":"I am",
	"im":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
	"mightnt":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
	"shouldnt":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
	"wasnt":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
	"werent":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
	"whos":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
	"wouldnt":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

In [9]:
def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [10]:
def remove_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [11]:
def preprocess_word(word):
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


In [12]:
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [13]:
def cleanText(text): 
    processed_tweet = []
    porter_stemmer = PorterStemmer()	
    text = text.strip().replace("\n", " ").replace("\r", " ")
    #Special case not handled previously.
    text = text.replace('\x92',"'")
    #Removal of address
    text = ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())	
    #text = replace_contractions(text)
    #text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    text = text.replace("’","'")
    words = text.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    text = " ".join(reformed)
    # Standardizing words
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = text.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    text = " ".join(reformed)
    #Deal with emojis
    text = emoji.demojize(text)
    #Strip accents
    text= strip_accents(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split())

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    words = text.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
             word = str(porter_stemmer.stem(word))
        processed_tweet.append(word)  
      
    return " ".join(words)

In [14]:
def tweet_label(label):
	if (label == "neutral"):return 0        # neutral
	elif (label == "worry"):return 1        # worry
	elif (label == "happiness"):return 2    # happiness
	elif (label == "sadness"):return 3      # sad
	elif (label == "love"):return 4         # love
	elif (label == "surprise"):return 5     # surprise
	elif (label == "fun"):return 6          # fun
	elif (label == "relief"):return 7       # relief
	elif (label == "hate"):return 8         # hate
	elif (label == "empty"):return 9        # empty
	elif (label == "enthusiasm"):return 10  # enthusiasm
	elif (label == "boredom"):return 11     # boredom
	elif (label == "anger"):return 12       # anger


#cleaning Process

In [15]:
crowdFlower = pd.read_csv('/content/croud_flower.csv',names = ['tweet_id','emotion','author','tweets'],skiprows = 1,header=None)

FileNotFoundError: ignored

In [16]:
try :
  crowdFlower.drop(['tweet_id','author'],axis=1,inplace=True)
except : 
  print("already removed")
crowdFlower.head()

already removed


NameError: ignored

In [ ]:
crowdFlower.to_csv('/content/gdrive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower.csv',index=None)

In [ ]:
crowdFlower['tweets'] = crowdFlower['tweets'].apply(lambda tweet : cleanText(tweet))
crowdFlower['emotion'] = crowdFlower['emotion'].apply(lambda label : tweet_label(label))

In [ ]:
crowdFlower.head()

In [ ]:
crowdFlower.to_csv('/content/gdrive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower_clean.csv')

#Spelling Correction Methods

In [88]:
def jaccard(word,gram_number):
  
  try:
    if dictionary[word]==1:
      return word
  except KeyError:
    spellings = spelling_series[spelling_series.str.startswith(word[:2])]

    distances = []
    if len(word) >=3: 
      distances = [(jaccard_distance(set(ngrams(word,3)),set(ngrams(x,3))),x) for x in spellings]
    elif len(word) == 2:
      distances = [(jaccard_distance(set(ngrams(word,2)),set(ngrams(x,2))),x) for x in spellings]
    
    closet = ("0",word)
    if len(distances)!=0:
      closet = min(distances)
    return closet[1]

In [87]:
def correct_spelling(word):
  return str(jaccard(word,3))


In [89]:
def correct_spelling_tweet(tweet):

  words = tweet.split()
  for i in range(len(words)):
    words[i] = correct_spelling(words[i])

  return " ".join(words)

In [91]:
vocab_glove = {}
spellings = []
dictionary = {}

In [92]:
with open("/content/gdrive/MyDrive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      dictionary[word] = 1
      spellings.append(word)
      vocab_glove[word] = vector

In [93]:
spelling_series = pd.Series(spellings)
print(spelling_series)

0            <user>
1                 .
2                 :
3                rt
4                 ,
             ...   
1193509     ﾊﾞｲﾊﾞｰｲ
1193510        ﾊﾟﾝﾁ
1193511       ﾔﾒﾀﾏｴ
1193512       ﾖｲｼｮｯ
1193513    ﾟﾟﾟｵﾔｽﾐｰ
Length: 1193514, dtype: object


In [1]:
 crowdFlower['tweets'] = crowdFlower['tweets'].apply(lambda x : correct_spelling_tweet(x))

NameError: ignored